## 하이브리드 검색

In [ ]:
#!pip install rank_bm25 #키워드 기반의 랭킹 알고리즘

  Using cached rank_bm25-0.2.2-py3-none-any.whl.metadata (3.2 kB)
Using cached rank_bm25-0.2.2-py3-none-any.whl (8.6 kB)


In [ ]:
# 0. 필요한 라이브러리 import
from langchain.retrievers import EnsembleRetriever # 여러 retriever를 조합하여 사용
from langchain_community.retrievers import BM25Retriever  # TF-IDF 계열의 키워드 기반 검색
from langchain_community.vectorstores import FAISS # Facebook의 벡터 유사도 검색 라이브러리
from langchain_openai import OpenAIEmbeddings # OpenAI 임베딩 모델

In [7]:
# 1. 환경 변수로부터 키값 가져오기
import os
from dotenv import load_dotenv

# 환경 변수 로드 및 OpenAI 클라이언트 설정
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [8]:
# 2. 첫 번째 데이터셋 준비 (강아지 성격 특성)
doc_list_1 = [
    "프렌치 불독: 사교적이고 친근한 성격을 가지고 있으며, 조용하고 집에서 지내기에 적합 합니다",
    "비글: 호기심이 많고, 에너지가 넘치며, 사냥 본능이 강합니다. ",
    "독일 셰퍼드: 용감하고 지능적이며, 충성심이 강합니다",
    "포메라니안: 활발하고 호기심이 많으며, 주인에게 매우 애정적입니다",
    "치와와: 작지만 용감하고, 주인에게 깊은 애정을 보입니다",
    "보더 콜리:	매우 지능적이고 학습 능력이 뛰어나며, 에너지가 많아 많은 운동이 필요합니다 "
]

# 3. 두 번째 데이터셋 준비 (강아지 관리 및 특이사항)
doc_list_2 = [
    "프렌치 불독: 열에 약하므로 주의가 필요합니다",
    "비글: 가족과 잘 지내며, 아이들과 노는 것을 좋아합니다.",
    "독일 셰퍼드: 경찰견이나 구조견으로 많이 활용되며, 적절한 훈련과 운동이 필요합니다.",
    "포메라니안: 털이 풍성하므로 정기적인 그루밍이 필요합니다.",
    "치와와: 다른 동물이나 낯선 사람에게는 조심스러울 수 있습니다.",
    "보더 콜리: 목축견으로서의 본능이 강하며, 다양한 트릭과 명령을 쉽게 배울 수 있습니다."
]

In [19]:
# 4. OpenAI 임베딩 모델 초기화 (벡터 검색용)
embedding = OpenAIEmbeddings(
    model="text-embedding-ada-002", 
    api_key=api_key  # 실제 API 키로 교체 필요
)

In [ ]:
# 5. BM25 Retriever 초기화 (키워드 기반 검색)
bm25_retriever = ... (             # ... code here
    doc_list_1, 
    metadatas=[{"source": 1}] * len(doc_list_1)  # 첫 번째 데이터셋 메타데이터
)
bm25_retriever.k = 2  # 상위 2개 문서 반환

# 6. FAISS Retriever 초기화 (벡터 유사도 기반 검색)
faiss_vectorstore = ...(     # ... code here
    doc_list_2, 
    embedding, 
    metadatas=[{"source": 2}] * len(doc_list_2)  # 두 번째 데이터셋 메타데이터
)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})

# 7. Ensemble Retriever 초기화 (두 검색 방법을 조합)
ensemble_retriever = ...(   # ... code here
    retrievers=[..., ...],   # ... code here
    weights=[0.5, 0.5]  # 각 retriever에 동일한 가중치 부여
)

In [16]:
# 8. 첫 번째 질문 테스트: "충성심이 강한 강아지는?"
query = "충성심이 강한 강아지는?"

bm25_result = bm25_retriever.get_relevant_documents(query)
faiss_result = faiss_retriever.get_relevant_documents(query)
ensemble_result = ensemble_retriever.get_relevant_documents(query)

# 첫 번째 질문 결과 출력
print("[BM25 Retriever - 키워드 검색]\n", bm25_result, end="\n\n")
print("[FAISS Retriever - 벡터 검색]\n", faiss_result, end="\n\n")
print("[Ensemble Retriever - 조합 검색]\n", ensemble_result, end="\n\n")

[BM25 Retriever - 키워드 검색]
 [Document(metadata={'source': 1}, page_content='독일 셰퍼드: 용감하고 지능적이며, 충성심이 강합니다'), Document(metadata={'source': 1}, page_content='보더 콜리:\t매우 지능적이고 학습 능력이 뛰어나며, 에너지가 많아 많은 운동이 필요합니다 ')]

[FAISS Retriever - 벡터 검색]
 [Document(id='eae765f1-82d5-441b-b782-0500c85ac454', metadata={'source': 2}, page_content='치와와: 다른 동물이나 낯선 사람에게는 조심스러울 수 있습니다.'), Document(id='3b32ca97-d4d6-44e6-8161-7713bca92402', metadata={'source': 2}, page_content='보더 콜리: 목축견으로서의 본능이 강하며, 다양한 트릭과 명령을 쉽게 배울 수 있습니다.')]

[Ensemble Retriever - 조합 검색]
 [Document(metadata={'source': 1}, page_content='독일 셰퍼드: 용감하고 지능적이며, 충성심이 강합니다'), Document(id='eae765f1-82d5-441b-b782-0500c85ac454', metadata={'source': 2}, page_content='치와와: 다른 동물이나 낯선 사람에게는 조심스러울 수 있습니다.'), Document(metadata={'source': 1}, page_content='보더 콜리:\t매우 지능적이고 학습 능력이 뛰어나며, 에너지가 많아 많은 운동이 필요합니다 '), Document(id='3b32ca97-d4d6-44e6-8161-7713bca92402', metadata={'source': 2}, page_content='보더 콜리: 목축견으로서의 본능이 강하며, 다양한 트릭과 명령을 쉽게 배울 수 있습니다.')]



In [17]:
# 9. 두 번째 질문 테스트: "지능적인 강아지는?"

query = "지능적인 강아지는?"

bm25_result = bm25_retriever.get_relevant_documents(query)
faiss_result = faiss_retriever.get_relevant_documents(query)
ensemble_result = ensemble_retriever.get_relevant_documents(query)

# 두 번째 질문 결과 출력

print("[BM25 Retriever - 키워드 검색]\n", bm25_result, end="\n\n")
print("[FAISS Retriever - 벡터 검색]\n", faiss_result, end="\n\n")
print("[Ensemble Retriever - 조합 검색]\n", ensemble_result, end="\n\n")

[BM25 Retriever - 키워드 검색]
 [Document(metadata={'source': 1}, page_content='보더 콜리:\t매우 지능적이고 학습 능력이 뛰어나며, 에너지가 많아 많은 운동이 필요합니다 '), Document(metadata={'source': 1}, page_content='치와와: 작지만 용감하고, 주인에게 깊은 애정을 보입니다')]

[FAISS Retriever - 벡터 검색]
 [Document(id='3b32ca97-d4d6-44e6-8161-7713bca92402', metadata={'source': 2}, page_content='보더 콜리: 목축견으로서의 본능이 강하며, 다양한 트릭과 명령을 쉽게 배울 수 있습니다.'), Document(id='eae765f1-82d5-441b-b782-0500c85ac454', metadata={'source': 2}, page_content='치와와: 다른 동물이나 낯선 사람에게는 조심스러울 수 있습니다.')]

[Ensemble Retriever - 조합 검색]
 [Document(metadata={'source': 1}, page_content='보더 콜리:\t매우 지능적이고 학습 능력이 뛰어나며, 에너지가 많아 많은 운동이 필요합니다 '), Document(id='3b32ca97-d4d6-44e6-8161-7713bca92402', metadata={'source': 2}, page_content='보더 콜리: 목축견으로서의 본능이 강하며, 다양한 트릭과 명령을 쉽게 배울 수 있습니다.'), Document(metadata={'source': 1}, page_content='치와와: 작지만 용감하고, 주인에게 깊은 애정을 보입니다'), Document(id='eae765f1-82d5-441b-b782-0500c85ac454', metadata={'source': 2}, page_content='치와와: 다른 동물이나 낯선 사람에게는 조심스러울 수 있습니다.')]